The datastore is a tool for using the eemeter which automates
and scales some of the most frequent tasks accomplished by the
eemeter. These tasks include data loading and storage, meter
running, and result storage and inspection. It puts a REST API
in front of the eemeter.

> Note:
> 
> For small and large datasets, the ETL toolkit exists to ease and
> speed up this process. That toolkit relies upon the API described
> in this tutorial and in the datastore API documentation. For the
> purpose of this tutorial, we will not be using the ETL toolkit.
> For more information on the ETL toolkit, see its API documentation.

Loading data
------------

For this tutorial, we will use the python requests package to make
requests to the datastore. We will use the same dataset used in the
eemeter tutorial, available for download here:

- project data CSV
- energy data CSV

This tutorial is also available as a jupyter notebook.

In [1]:
# library imports
import pandas as pd
import requests
import pytz

If you followed the datastore development setup instructions, you will
already have run the command to create a superuser and access credentials.

    python manage.py dev_seed
    
If you haven't already done so, do so now. The `dev_seed` command
creates a demo admin user and a sample project.

- username: `demo`,
- password: `demo-password`,
- API access token: `tokstr`.

Ensure that your development server is running locally on port 8000 before continuing.

    python manage.py runserver
    
Each request will include an Authorization header

    Authorization: Bearer tokstr

In [2]:
base_url = "http://0.0.0.0:8000"
headers = {"Authorization": "Bearer tokstr"}

Let's test the API by requesting a list of projects in the datastore. Since the dev_seed command creates a sample project, this will return a response showing that project.

In [3]:
url = base_url + "/api/v1/projects/"
projects = requests.get(url, headers=headers).json()

In [4]:
projects

[{'baseline_period_end': '2012-01-01T00:00:00Z',
  'baseline_period_start': None,
  'id': 1,
  'project_id': 'ABC',
  'project_owner': 1,
  'reporting_period_end': None,
  'reporting_period_start': '2012-02-01T00:00:00Z',
  'zipcode': '91104'}]

Although we'll delete this one in a moment, we can first explore a
bit to get a feel for the API. Then we'll create a project of our own.

Energy trace data will be associated with this project by foreign key.
It is organized into time series by trace_id, and the following request
will show all traces associated with a particular project. Note the
difference between the `'id'` field and the `'project_id'` field.
The `'project_id'` field is the unique label that was associated with
it by an external source; the `'id'` field is the database table primary
key.

Let's inspect the traces associated with this project. We can do so
using the project primary key `'id'` as a filter (we use the summary flag so that we
don't pull every record):

In [5]:
url = base_url + "/api/v1/consumption_metadatas/?summary=True&projects={}".format(projects[0]['id'])
consumption_metadatas = requests.get(url, headers=headers).json()

In [6]:
consumption_metadatas[0]

{'id': 1,
 'interpretation': 'NG_C_S',
 'label': None,
 'project': {'baseline_period_end': '2012-01-01T00:00:00Z',
  'baseline_period_start': None,
  'id': 1,
  'project_id': 'ABC',
  'project_owner': 1,
  'reporting_period_end': None,
  'reporting_period_start': '2012-02-01T00:00:00Z',
  'zipcode': '91104'},
 'unit': 'THM'}

We can also query for consumption records by metadata primary key.

In [7]:
url = base_url + "/api/v1/consumption_records/?metadata={}".format(consumption_metadatas[0]['id'])
consumption_records = requests.get(url, headers=headers).json()

In [8]:
consumption_records[:3]

[{'estimated': False,
  'id': 1,
  'metadata': 1,
  'start': '2010-01-01T00:00:00Z',
  'value': None},
 {'estimated': False,
  'id': 2,
  'metadata': 1,
  'start': '2010-02-01T00:00:00Z',
  'value': 1.0},
 {'estimated': False,
  'id': 3,
  'metadata': 1,
  'start': '2010-03-01T00:00:00Z',
  'value': 1.0}]

Now we'll delete the project that was created by the dev_seed command and make one of our own.

In [9]:
url = base_url + "/api/v1/projects/{}/".format(projects[0]['id'])
requests.delete(url, headers=headers)

<Response [204]>

In [10]:
project_data = pd.read_csv('sample-project-data.csv',
                           parse_dates=['retrofit_start_date', 'retrofit_end_date']).iloc[0]

In [11]:
project_data

project_id                             ABC
zipcode                              50321
retrofit_start_date    2013-06-01 00:00:00
retrofit_end_date      2013-07-01 00:00:00
Name: 0, dtype: object

In [12]:
data = {
    "project_id": project_data.project_id,
    "zipcode": str(project_data.zipcode),
    "baseline_period_end": pytz.UTC.localize(project_data.retrofit_start_date).isoformat(),
    "reporting_period_start": pytz.UTC.localize(project_data.retrofit_end_date).isoformat(),
    "project_owner": 1,  # ID of user created by dev_seed command
}
print(data)

{'reporting_period_start': '2013-07-01T00:00:00+00:00', 'project_id': 'ABC', 'zipcode': '50321', 'project_owner': 1, 'baseline_period_end': '2013-06-01T00:00:00+00:00'}


In [13]:
url = base_url + "/api/v1/projects/"
new_project = requests.post(url, json=data, headers=headers).json()
new_project

{'baseline_period_end': '2013-06-01T00:00:00Z',
 'baseline_period_start': None,
 'id': 2,
 'project_id': 'ABC',
 'project_owner': 1,
 'reporting_period_end': None,
 'reporting_period_start': '2013-07-01T00:00:00Z',
 'zipcode': '50321'}

If you try to post another project with the same `project_id`, you'll get an error message.

In [14]:
url = base_url + "/api/v1/projects/"
requests.post(url, json=data, headers=headers).json()

{'project_id': ['project with this project id already exists.']}

Now we can give this project some consumption data.

In [15]:
energy_data = pd.read_csv('sample-energy-data_project-ABC_zipcode-50321.csv',
                          parse_dates=['date'], dtype={'zipcode': str})

In [29]:
energy_data.head()

,project_id,trace_id,date,value,unit,fuel,estimated
0,ABC,DEF,2011-01-01,57.8,kWh,electricity,False
1,ABC,DEF,2011-01-02,64.8,kWh,electricity,False
2,ABC,DEF,2011-01-03,49.5,kWh,electricity,False
3,ABC,DEF,2011-01-04,80.0,kWh,electricity,False
4,ABC,DEF,2011-01-05,51.7,kWh,electricity,False


In [16]:
records = [{
    "start": pytz.UTC.localize(row.date.to_datetime()).isoformat(),
    "value": row.value,
    "estimated": row.estimated,
} for _, row in energy_data.iterrows()]

We'll hit the `sync` endpoint for consumption metadata, which will create a new record or update an existing record. We have one project here:

In [36]:
interpretation_mapping = {"electricity": "E_C_S"}
data = [
    {
        "project_project_id": energy_data.iloc[0]["project_id"],
        "interpretation": [energy_data.iloc[0]["fuel"]],
        "unit": energy_data.iloc[0]["unit"].upper(),
        "label": energy_data.iloc[0]["trace_id"].upper()
    }
]
data

[{'interpretation': ['electricity'],
  'label': 'DEF',
  'project_project_id': 'ABC',
  'unit': 'KWH'}]

In [27]:
url = base_url + "/api/v1/consumption_metadatas/sync/"
consumption_metadatas = requests.post(url, json=data, headers=headers).json()

In [28]:
consumption_metadatas

[{'id': 7,
  'interpretation': 'E_C_S',
  'label': 'DEF',
  'project': {'baseline_period_end': '2013-06-01T00:00:00Z',
   'baseline_period_start': None,
   'id': 2,
   'project_id': 'ABC',
   'project_owner': 1,
   'reporting_period_end': None,
   'reporting_period_start': '2013-07-01T00:00:00Z',
   'zipcode': '50321'},
  'status': 'unchanged - same record',
  'unit': 'KWH'}]